ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :


В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

# Загрузка данных

In [ ]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [ ]:
ratings.shape

(100836, 4)

In [ ]:
movies = pd.read_csv('movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
movies.shape

(9742, 3)

In [ ]:
df = ratings.merge(movies)
print(df.shape)
df.head()

(100836, 6)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


# Загрузка SURPRISE

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366563 sha256=882c003828a9ade85010f16438ce395b05edec0a8615c7c0ad3db45b857d426d
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans, KNNBasic

reader = Reader(rating_scale=(df['rating'].min(), df['rating'].max()))
data = Dataset.load_from_df(df[['userId',	'movieId', 'rating']], reader)

# KNNWithMeans

In [ ]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
cv = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
cv['test_rmse'].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8727  0.8900  0.8892  0.8757  0.8889  0.8833  0.0075  
Fit time          8.72    7.87    7.72    9.70    8.39    8.48    0.71    
Test time         9.11    10.11   10.08   10.08   9.00    9.68    0.51    


0.8833077880711034

# KNNBasic

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 40,
               }
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(bsl_options=bsl_options, sim_options=sim_options)

cv = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
cv['test_rmse'].mean()

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.9725226680152709

# CoClustering

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.co_clustering import CoClustering

param_grid = {'n_cltr_u': range(1,7),
              'n_cltr_i': range(1,7),
              'n_epochs': [10, 20, 40],
              'random_state': [123],
              }
gs = GridSearchCV(CoClustering, param_grid, measures=['rmse'], cv=5, joblib_verbose=2)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


0.9364590366035179
{'n_cltr_u': 1, 'n_cltr_i': 1, 'n_epochs': 10, 'random_state': 123}


[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed: 39.3min finished


# SVD

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.matrix_factorization import SVD

param_grid = {'n_factors': [50, 100, 200, 300],
              'n_epochs': [10, 20, 40, 50],
              'lr_all': [0.001, 0.005, 0.01, 0.05],
              'reg_all': [0.02],
              'random_state': [123],
              }
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8591574554684718
{'n_factors': 300, 'n_epochs': 40, 'lr_all': 0.05, 'reg_all': 0.02, 'random_state': 123}
